In [1]:
%load_ext autoreload

%autoreload 2

### Run finetuning

* Prepare your files and split to shards. All files will be splitted between all gpus. All paths to text files should be stored at file.list

Example of `files.list` for single gpu:

```text
data/train.txt
```

* Add your path to template script in our repo [scripts/deepspeed_gpt3_xl_finetune.sh](https://github.com/sberbank-ai/ru-gpts/blob/master/scripts/deepspeed_gpt3_xl_finetune.sh)

* `cd scripts` for go to scripts dir

* run command

```bash
sh deepspeed_gpt3_xl_finetune.sh

```

### Test model

#### Load model

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../../ru-gpts/")

In [4]:
import os
os.environ["USE_DEEPSPEED"] = "1"

In [ ]:
from src.xl_wrapper import RuGPT3XL

Note! seq_len is max sequence length for generation used in generation process. Max avialable seq_len is 2048 (in tokens).
Also inference takes around 10 Gb GPU memory.

In [6]:
gpt = RuGPT3XL.from_pretrained(
    "sberbank-ai/rugpt3xl",
    weights_path="/path/2/save/model/train-iters/mp_rank_00_model_states.pt",
    deepspeed_config_path="../src/deepspeed_config/gpt3_xl_sparse_2048.json",
    seq_len=512
)

> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234


/home/user/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Use alternating sparse & dense attention layers


#### Example of generation

In [13]:
def generate(text, additional_len=20):
    return filter_resuls(gpt.generate(
        text=text,
        max_length=len(gpt.tokenizer.encode(text)) + additional_len,
        num_beams=10,
        eos_token_id=gpt.tokenizer.eos_token_id,
        num_return_sequences=1,
    ))[0]

In [15]:
def filter_resuls(nr):
    return [x[:x.find("<|endoftext|>")] for x in nr]

In [23]:
generate("Фрегат «Адмирал Макаров»")

'Фрегат «Адмирал Макаров» вошел в Средиземное море\u2028Большой противолодочный корабль «Вице-адмирал Кулак'